In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
pd.set_option('display.max_columns', 500)

In [66]:
os.listdir('Raw')

['CTNNB1.csv',
 'LEF1.csv',
 'Jnk.csv',
 'Erk.csv',
 'IRF4.csv',
 'MYC.csv',
 'Ikk2.csv']

In [3]:
from utils import load_fold
train, test, test_fold = load_fold('Raw', 5)

Test fold: MYC.csv
Train fold: CTNNB1.csv
Train fold: LEF1.csv
Train fold: Jnk.csv
Train fold: Erk.csv
Train fold: IRF4.csv
Train fold: Ikk2.csv


In [8]:
from utils import outer_product
from utils import group_data
X_train, y_train = group_data(outer_product(train))
X_test, y_test = group_data(outer_product(test))

In [6]:
X_train.shape, X_test.shape

((118263, 6, 81), (10667, 6, 81))

In [21]:
from utils import sklearn_train_test_split
X_tra, X_val, y_tra, y_val = sklearn_train_test_split(
                X_train, y_train, 0.1, random_state=42,
                shuffle=True, stratify=y_train)

In [22]:
X_tra.shape, X_val.shape, y_tra.shape, y_val.shape

((106436, 6, 81), (11827, 6, 81), (106436, 6), (11827, 6))

## CNN

In [26]:
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import BatchNormalization
from keras.layers import Activation

/home/raduviri/anaconda3/envs/chana/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [27]:
def create_model():
    model = Sequential()

    model.add(Conv2D(16, (3, 3), padding='same', input_shape=(9, 9, 1)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Conv2D(32, (3, 3), strides=2))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Conv2D(32, (1, 1)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Flatten())
    
    model.add(Dense(32))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
       
#     model.add(GlobalAveragePooling2D())

    model.add(Dense(1, activation='sigmoid'))
    
    adam = Adam()
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [28]:
create_model().summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 9, 9, 16)          160       
_________________________________________________________________
activation_1 (Activation)    (None, 9, 9, 16)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 9, 16)          64        
_________________________________________________________________
dropout_1 (Dropout)          (None, 9, 9, 16)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 32)          4640      
_________________________________________________________________
activation_2 (Activation)    (None, 4, 4, 32)          0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 4, 4, 32)          128       
__________

In [29]:
model = KerasClassifier(create_model)

In [31]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='_data/model.best.hdf5',
                               verbose=1, save_best_only=True)

In [30]:
X_tra.shape, X_val.shape, y_tra.shape, y_val.shape

((106436, 6, 81), (11827, 6, 81), (106436, 6), (11827, 6))

In [35]:
sum(y_tra), sum(y_val)

(array([1827, 1827, 1827, 1827, 1827, 1827]),
 array([203, 203, 203, 203, 203, 203]))

In [36]:
from utils import get_random_batch
from imblearn.under_sampling import RandomUnderSampler

In [42]:
rus = RandomUnderSampler(random_state=0)
X_train_batch, y_train_batch = get_random_batch(X_tra, y_tra, rus)
X_val_batch, y_val_batch = get_random_batch(X_val, y_val, rus)

In [44]:
X_train_batch.shape, X_tra.shape

((3654, 6, 81), (106436, 6, 81))

In [43]:
history = model.fit(X_train_batch.reshape(-1, 9, 9, 1), 
                    y_train_batch.reshape(-1), 
                     validation_data=(X_val_batch.reshape(-1, 9, 9, 1),
                                      y_val_batch.reshape(-1)),
                     batch_size=128, epochs=100, verbose=1, 
                     callbacks=[checkpointer], shuffle=True)

Train on 21924 samples, validate on 2436 samples
Epoch 1/100
21504/21924 [============================>.] - ETA: 0s - loss: 0.8524 - acc: 0.5162
Epoch 00001: val_loss did not improve
21924/21924 [==============================] - 3s 118us/step - loss: 0.8516 - acc: 0.5161 - val_loss: 0.6802 - val_acc: 0.5489
Epoch 2/100
21888/21924 [============================>.] - ETA: 0s - loss: 0.7372 - acc: 0.5344
Epoch 00002: val_loss did not improve
21924/21924 [==============================] - 2s 70us/step - loss: 0.7371 - acc: 0.5344 - val_loss: 0.6822 - val_acc: 0.5435
Epoch 3/100
21888/21924 [============================>.] - ETA: 0s - loss: 0.6981 - acc: 0.5491
Epoch 00003: val_loss did not improve
21924/21924 [==============================] - 2s 77us/step - loss: 0.6981 - acc: 0.5491 - val_loss: 0.6805 - val_acc: 0.5493
Epoch 4/100
21504/21924 [============================>.] - ETA: 0s - loss: 0.6819 - acc: 0.5656
Epoch 00004: val_loss did not improve
21924/21924 [=======================

21504/21924 [============================>.] - ETA: 0s - loss: 0.5377 - acc: 0.7379
Epoch 00032: val_loss did not improve
21924/21924 [==============================] - 2s 69us/step - loss: 0.5376 - acc: 0.7375 - val_loss: 0.5186 - val_acc: 0.7344
Epoch 33/100
21248/21924 [============================>.] - ETA: 0s - loss: 0.5344 - acc: 0.7388
Epoch 00033: val_loss did not improve
21924/21924 [==============================] - 2s 78us/step - loss: 0.5339 - acc: 0.7391 - val_loss: 0.5220 - val_acc: 0.7373
Epoch 34/100
21248/21924 [============================>.] - ETA: 0s - loss: 0.5328 - acc: 0.7430
Epoch 00034: val_loss did not improve
21924/21924 [==============================] - 2s 80us/step - loss: 0.5337 - acc: 0.7418 - val_loss: 0.5151 - val_acc: 0.7410
Epoch 35/100
21760/21924 [============================>.] - ETA: 0s - loss: 0.5308 - acc: 0.7411
Epoch 00035: val_loss did not improve
21924/21924 [==============================] - 1s 64us/step - loss: 0.5306 - acc: 0.7414 - val_

Epoch 64/100
21760/21924 [============================>.] - ETA: 0s - loss: 0.5035 - acc: 0.7631
Epoch 00064: val_loss did not improve
21924/21924 [==============================] - 2s 76us/step - loss: 0.5035 - acc: 0.7631 - val_loss: 0.4940 - val_acc: 0.7545
Epoch 65/100
21632/21924 [============================>.] - ETA: 0s - loss: 0.5062 - acc: 0.7643
Epoch 00065: val_loss did not improve
21924/21924 [==============================] - 1s 68us/step - loss: 0.5060 - acc: 0.7645 - val_loss: 0.4956 - val_acc: 0.7566
Epoch 66/100
21248/21924 [============================>.] - ETA: 0s - loss: 0.5076 - acc: 0.7618
Epoch 00066: val_loss did not improve
21924/21924 [==============================] - 2s 75us/step - loss: 0.5084 - acc: 0.7615 - val_loss: 0.4959 - val_acc: 0.7516
Epoch 67/100
21376/21924 [============================>.] - ETA: 0s - loss: 0.5078 - acc: 0.7612
Epoch 00067: val_loss did not improve
21924/21924 [==============================] - 2s 74us/step - loss: 0.5073 - acc: 

Epoch 96/100
21376/21924 [============================>.] - ETA: 0s - loss: 0.5010 - acc: 0.7658
Epoch 00096: val_loss did not improve
21924/21924 [==============================] - 2s 79us/step - loss: 0.5009 - acc: 0.7664 - val_loss: 0.4931 - val_acc: 0.7607
Epoch 97/100
21632/21924 [============================>.] - ETA: 0s - loss: 0.5014 - acc: 0.7638
Epoch 00097: val_loss did not improve
21924/21924 [==============================] - 2s 74us/step - loss: 0.5015 - acc: 0.7639 - val_loss: 0.4873 - val_acc: 0.7623
Epoch 98/100
21504/21924 [============================>.] - ETA: 0s - loss: 0.4966 - acc: 0.7679
Epoch 00098: val_loss did not improve
21924/21924 [==============================] - 1s 60us/step - loss: 0.4970 - acc: 0.7673 - val_loss: 0.4890 - val_acc: 0.7611
Epoch 99/100
21376/21924 [============================>.] - ETA: 0s - loss: 0.4985 - acc: 0.7671
Epoch 00099: val_loss did not improve
21924/21924 [==============================] - 2s 74us/step - loss: 0.4984 - acc: 

In [54]:
y_pred = model.predict(X_val_batch.reshape(-1, 9, 9, 1))
y_pred_proba = model.predict_proba(X_val_batch.reshape(-1, 9, 9, 1))

In [55]:
from utils import evaluate_metrics

In [58]:
evaluate_metrics(y_val_batch.reshape(-1), y_pred, y_pred_proba[:,1])

Confusion matrix:
[[ 723  495]
 [  93 1125]]
  Model Fold   ROC-AUC    G-mean  F1-Score       TPR       TNR  Accuracy  \
0             0.837806  0.740454  0.792812  0.923645  0.593596  0.758621   

   Precision    Recall   Logloss Description  
0   0.694444  0.923645  0.490929              


,Model,Fold,ROC-AUC,G-mean,F1-Score,TPR,TNR,Accuracy,Precision,Recall,Logloss,Description
0,,,0.837806,0.740454,0.792812,0.923645,0.593596,0.758621,0.694444,0.923645,0.490929,


In [59]:
X_test.shape, y_test.shape

((10667, 6, 81), (10667, 6))

In [61]:
y_pred = model.predict(X_test.reshape(-1, 9, 9, 1))
y_pred_proba = model.predict_proba(X_test.reshape(-1, 9, 9, 1))

In [63]:
evaluate_metrics(y_test.reshape(-1), y_pred, y_pred_proba[:,1])

Confusion matrix:
[[16535 41869]
 [ 1135  4463]]
  Model Fold   ROC-AUC    G-mean  F1-Score       TPR       TNR  Accuracy  \
0             0.633002  0.475092  0.171885  0.797249  0.283114  0.328083   

   Precision    Recall   Logloss Description  
0   0.096327  0.797249  0.905357              


,Model,Fold,ROC-AUC,G-mean,F1-Score,TPR,TNR,Accuracy,Precision,Recall,Logloss,Description
0,,,0.633002,0.475092,0.171885,0.797249,0.283114,0.328083,0.096327,0.797249,0.905357,


In [64]:
test_fold

'MYC.csv'

## LEF1

In [67]:
from utils import load_fold
train, test, test_fold = load_fold('Raw', 1)

Test fold: LEF1.csv
Train fold: CTNNB1.csv
Train fold: Jnk.csv
Train fold: Erk.csv
Train fold: IRF4.csv
Train fold: MYC.csv
Train fold: Ikk2.csv


In [68]:
from utils import outer_product
from utils import group_data
X_train, y_train = group_data(outer_product(train))
X_test, y_test = group_data(outer_product(test))

In [69]:
from utils import sklearn_train_test_split
X_tra, X_val, y_tra, y_val = sklearn_train_test_split(
                X_train, y_train, 0.1, random_state=42,
                shuffle=True, stratify=y_train)

In [70]:
model = KerasClassifier(create_model)

In [71]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='_data/model.best.hdf5',
                               verbose=1, save_best_only=True)

In [72]:
rus = RandomUnderSampler(random_state=0)
X_train_batch, y_train_batch = get_random_batch(X_tra, y_tra, rus)
X_val_batch, y_val_batch = get_random_batch(X_val, y_val, rus)

In [73]:
X_train_batch.shape, X_tra.shape

((5294, 6, 81), (105826, 6, 81))

In [75]:
history = model.fit(X_train_batch.reshape(-1, 9, 9, 1), 
                    y_train_batch.reshape(-1), 
                     validation_data=(X_val_batch.reshape(-1, 9, 9, 1),
                                      y_val_batch.reshape(-1)),
                     batch_size=128, epochs=100, verbose=1, 
                     callbacks=[checkpointer], shuffle=True)

Train on 31764 samples, validate on 3528 samples
Epoch 1/100
31104/31764 [============================>.] - ETA: 0s - loss: 0.8291 - acc: 0.5202
Epoch 00001: val_loss improved from inf to 0.70187, saving model to _data/model.best.hdf5
31764/31764 [==============================] - 4s 124us/step - loss: 0.8271 - acc: 0.5207 - val_loss: 0.7019 - val_acc: 0.5099
Epoch 2/100
30848/31764 [============================>.] - ETA: 0s - loss: 0.7066 - acc: 0.5519
Epoch 00002: val_loss improved from 0.70187 to 0.69145, saving model to _data/model.best.hdf5
31764/31764 [==============================] - 2s 68us/step - loss: 0.7059 - acc: 0.5530 - val_loss: 0.6915 - val_acc: 0.5193
Epoch 3/100
31616/31764 [============================>.] - ETA: 0s - loss: 0.6774 - acc: 0.5852
Epoch 00003: val_loss improved from 0.69145 to 0.66952, saving model to _data/model.best.hdf5
31764/31764 [==============================] - 2s 77us/step - loss: 0.6774 - acc: 0.5851 - val_loss: 0.6695 - val_acc: 0.5570
Epoch 

Epoch 28/100
31104/31764 [============================>.] - ETA: 0s - loss: 0.5385 - acc: 0.7263
Epoch 00028: val_loss did not improve
31764/31764 [==============================] - 2s 74us/step - loss: 0.5379 - acc: 0.7265 - val_loss: 0.4813 - val_acc: 0.7628
Epoch 29/100
31232/31764 [============================>.] - ETA: 0s - loss: 0.5330 - acc: 0.7301
Epoch 00029: val_loss improved from 0.47947 to 0.47504, saving model to _data/model.best.hdf5
31764/31764 [==============================] - 3s 79us/step - loss: 0.5332 - acc: 0.7297 - val_loss: 0.4750 - val_acc: 0.7642
Epoch 30/100
30976/31764 [============================>.] - ETA: 0s - loss: 0.5338 - acc: 0.7298
Epoch 00030: val_loss improved from 0.47504 to 0.47380, saving model to _data/model.best.hdf5
31764/31764 [==============================] - 2s 74us/step - loss: 0.5336 - acc: 0.7302 - val_loss: 0.4738 - val_acc: 0.7676
Epoch 31/100
31360/31764 [============================>.] - ETA: 0s - loss: 0.5312 - acc: 0.7309
Epoch 00

Epoch 58/100
31744/31764 [============================>.] - ETA: 0s - loss: 0.5114 - acc: 0.7456
Epoch 00058: val_loss did not improve
31764/31764 [==============================] - 2s 70us/step - loss: 0.5114 - acc: 0.7456 - val_loss: 0.4705 - val_acc: 0.7673
Epoch 59/100
31488/31764 [============================>.] - ETA: 0s - loss: 0.5180 - acc: 0.7424
Epoch 00059: val_loss did not improve
31764/31764 [==============================] - 2s 78us/step - loss: 0.5178 - acc: 0.7427 - val_loss: 0.4689 - val_acc: 0.7704
Epoch 60/100
31744/31764 [============================>.] - ETA: 0s - loss: 0.5151 - acc: 0.7450
Epoch 00060: val_loss did not improve
31764/31764 [==============================] - 2s 70us/step - loss: 0.5151 - acc: 0.7450 - val_loss: 0.4706 - val_acc: 0.7696
Epoch 61/100
31232/31764 [============================>.] - ETA: 0s - loss: 0.5141 - acc: 0.7447
Epoch 00061: val_loss did not improve
31764/31764 [==============================] - 2s 70us/step - loss: 0.5138 - acc: 

31232/31764 [============================>.] - ETA: 0s - loss: 0.5086 - acc: 0.7482
Epoch 00088: val_loss did not improve
31764/31764 [==============================] - 2s 66us/step - loss: 0.5089 - acc: 0.7479 - val_loss: 0.4670 - val_acc: 0.7690
Epoch 89/100
31232/31764 [============================>.] - ETA: 0s - loss: 0.5073 - acc: 0.7484
Epoch 00089: val_loss did not improve
31764/31764 [==============================] - 2s 65us/step - loss: 0.5075 - acc: 0.7483 - val_loss: 0.4673 - val_acc: 0.7696
Epoch 90/100
31232/31764 [============================>.] - ETA: 0s - loss: 0.5077 - acc: 0.7493
Epoch 00090: val_loss did not improve
31764/31764 [==============================] - 2s 67us/step - loss: 0.5079 - acc: 0.7494 - val_loss: 0.4677 - val_acc: 0.7679
Epoch 91/100
30848/31764 [============================>.] - ETA: 0s - loss: 0.5050 - acc: 0.7520
Epoch 00091: val_loss did not improve
31764/31764 [==============================] - 2s 72us/step - loss: 0.5064 - acc: 0.7514 - val_

In [76]:
y_pred = model.predict(X_val_batch.reshape(-1, 9, 9, 1))
y_pred_proba = model.predict_proba(X_val_batch.reshape(-1, 9, 9, 1))
evaluate_metrics(y_val_batch.reshape(-1), y_pred, y_pred_proba[:,1])

Confusion matrix:
[[1130  634]
 [ 169 1595]]
  Model Fold   ROC-AUC    G-mean  F1-Score       TPR      TNR  Accuracy  \
0             0.856242  0.761064  0.798898  0.904195  0.64059  0.772392   

   Precision    Recall   Logloss Description  
0   0.715568  0.904195  0.464701              


,Model,Fold,ROC-AUC,G-mean,F1-Score,TPR,TNR,Accuracy,Precision,Recall,Logloss,Description
0,,,0.856242,0.761064,0.798898,0.904195,0.64059,0.772392,0.715568,0.904195,0.464701,


In [77]:
y_pred = model.predict(X_test.reshape(-1, 9, 9, 1))
y_pred_proba = model.predict_proba(X_test.reshape(-1, 9, 9, 1))
evaluate_metrics(y_test.reshape(-1), y_pred, y_pred_proba[:,1])

Confusion matrix:
[[36872 31066]
 [   33    99]]
  Model Fold  ROC-AUC    G-mean  F1-Score   TPR      TNR  Accuracy  Precision  \
0             0.68041  0.638003  0.006326  0.75  0.54273  0.543132   0.003177   

   Recall   Logloss Description  
0    0.75  0.842144              


,Model,Fold,ROC-AUC,G-mean,F1-Score,TPR,TNR,Accuracy,Precision,Recall,Logloss,Description
0,,,0.68041,0.638003,0.006326,0.75,0.54273,0.543132,0.003177,0.75,0.842144,


In [ ]:
test_fold